In [11]:
import gzip
import os
import hashlib
import tkinter as tk
from tkinter import filedialog
import csv

class LogConverterGUI:
    def __init__(self, master):
        self.master = master
        master.title("Log Converter")


        # Create input file selection button
        self.input_files_button = tk.Button(master, text="Select input files", command=self.select_input_files)
        self.input_files_button.pack(padx=10, pady=10)

        # Create output file selection button
        self.output_file_button = tk.Button(master, text="Select output file", command=self.select_output_file)
        self.output_file_button.pack(padx=10, pady=10)

        # Create conversion button
        self.convert_button = tk.Button(master, text="Convert", command=self.convert)
        self.convert_button.pack(padx=10, pady=10)
        

        # Initialize file selection variables
        self.input_files = []
        self.output_file = ""
        self.master.geometry("500x200")
        #title of the window
        self.master.title("Network Ip Address updater")
        
        # quit button
        self.quitButton = tk.Button(self.master, text = 'Quit', width = 25, command = self.master.destroy)
        self.quitButton.pack()
        

    def select_input_files(self):
        # Open file dialog to select input files
        filenames = filedialog.askopenfilenames(title="Select input files", filetypes=(("Gzip files", "*.gz"), ("All files", "*.*")))

        # Update input file list
        self.input_files = list(filenames)

    def select_output_file(self):
        # Open file dialog to select output file
        filename = filedialog.asksaveasfilename(title="Select output file", defaultextension=".csv", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))

        # Update output file
        self.output_file = filename

    def convert(self):
        # Convert each input file to the output file
        if len(self.input_files) == 0:
            tk.messagebox.showerror("Error", "Please select at least one input file.")
            return

        if self.output_file == "":
            tk.messagebox.showerror("Error", "Please select an output file.")
            return

        header_written = False 
        processed_rows = set()

        if os.path.exists(self.output_file):
            mode = 'a' # append if file exists
            header_written = True
        
            with open(self.output_file, 'r') as f:
                reader = csv.reader(f)
                for row in reader:
                    processed_rows.add(hash_row(row))

        else:
            mode = 'w' # write if file does not exist

        with open(self.output_file, mode, newline='') as f_output:
            csv_writer = csv.writer(f_output, delimiter=',')

            if not header_written:
                csv_writer.writerow(["version", "account-id", "interface-id", "srcaddr", "dstaddr", "srcport", "dstport", "protocol", "packets", "bytes", "start", "end", "action", "log-status"])
                header_written = True

            for input_file in self.input_files:
                with gzip.open(input_file, 'rt') as f_input:
                    for line in f_input:
                        data = []
                        for item in line.split(" "):
                            if item != "":
                                data.append(item)
                        row_hash = hash_row(data) 
                        if row_hash not in processed_rows:
                            csv_writer.writerow(data)
                            processed_rows.add(row_hash)

        tk.messagebox.showinfo("Success", "Conversion complete.")

def hash_row(row):
    return hashlib.sha1(str(row).encode()).hexdigest()

root = tk.Tk()
app = LogConverterGUI(root)
root.mainloop()


In [1]:
import gzip
import os
import hashlib
import tkinter as tk
from tkinter import filedialog
import csv

class LogConverterGUI:
    def __init__(self, master):
        self.master = master
        master.title("Log Converter")


        # Create input file selection button
        self.input_files_button = tk.Button(master, text="Select input files", command=self.select_input_files)
        self.input_files_button.pack(padx=10, pady=10)

        # Create output file selection button
        self.output_file_button = tk.Button(master, text="Select output file", command=self.select_output_file)
        self.output_file_button.pack(padx=10, pady=10)

        # Create conversion button
        self.convert_button = tk.Button(master, text="Convert", command=self.convert)
        self.convert_button.pack(padx=10, pady=10)

        # Initialize file selection variables
        self.input_files = []
        self.output_file = ""
        self.master.geometry("500x200")
        #title of the window
        self.master.title("Network Ip Address updater")
        
        # quit button
        self.quitButton = tk.Button(self.master, text = 'Quit', width = 25, command = self.master.destroy)
        self.quitButton.pack()

    def select_input_files(self):
        # Open file dialog to select input files
        filenames = filedialog.askopenfilenames(title="Select input files", filetypes=(("Gzip files", "*.gz"), ("All files", "*.*")))

        # Update input file list
        self.input_files = list(filenames)

    def select_output_file(self):
        # Open file dialog to select output file
        filename = filedialog.asksaveasfilename(title="Select output file", defaultextension=".csv", filetypes=(("CSV files", "*.csv"), ("All files", "*.*")))

        # Update output file
        self.output_file = filename

    def convert(self):
        # Convert each input file to the output file
        if len(self.input_files) == 0:
            tk.messagebox.showerror("Error", "Please select at least one input file.")
            return

        if self.output_file == "":
            tk.messagebox.showerror("Error", "Please select an output file.")
            return

        processed_rows = set()

        if os.path.exists(self.output_file):
            mode = 'a' # append if file exists
            with open(self.output_file, 'r') as f:
                reader = csv.reader(f)
                for i, row in enumerate(reader):
                    if i == 0:
                        header_row = row
                    else:
                        processed_rows.add(hash_row(row))
        else:
            mode = 'w' # write if file does not exist
            header_row = ["version", "account-id", "interface-id", "srcaddr", "dstaddr", "srcport", "dstport", "protocol", "packets", "bytes", "start", "end", "action", "log-status"]

        with open(self.output_file, mode, newline='') as f_output:
            csv_writer = csv.writer(f_output, delimiter=',')

            if mode == 'w':
                csv_writer.writerow(header_row)

            for input_file in self.input_files:
                with gzip.open(input_file, 'rt') as f_input:
                    for line in f_input:
                        data = []
                        for item in line.split(" "):
                            if item != "":
                                data.append(item)
                        row_hash = hash_row(data) 
                        if row_hash not in processed_rows:
                            csv_writer.writerow(data)
                            processed_rows.add(row_hash)

        tk.messagebox.showinfo("Success", "Conversion complete.")


def hash_row(row):
    return hashlib.sha1(str(row).encode()).hexdigest()

root = tk.Tk()
app = LogConverterGUI(root)
root.mainloop()


In [1]:
import tkinter as tk
from tkinter import filedialog
import csv
import hashlib

# Function to browse and select the csv files
def browse_files():
    file_path = filedialog.askopenfilename()
    path_entry.delete(0, tk.END)
    path_entry.insert(0, file_path)

# Function to convert csv file to its hash value
def convert():
    try:
        filename = path_entry.get()
        with open(filename, 'rb') as file:
            file_content = file.read()
            hash_value = hashlib.md5(file_content).hexdigest()
        with open('hash_value.txt', 'w') as file:
            file.write(hash_value)

        with open(filename, 'r') as input_file:
            reader = csv.reader(input_file)
            header = next(reader)
            data = []
            for row in reader:
                data.append(row)

        with open('output.csv', 'a', newline='') as output_file:
            writer = csv.writer(output_file)
            # If output file is empty, add header
            output_file.seek(0)
            first_char = output_file.read(1)
            if not first_char:
                writer.writerow(header)
            for row in data:
                writer.writerow(row)
        status_label.config(text="Conversion Successful!", fg="green")
    except Exception as e:
        status_label.config(text="Conversion Failed: " + str(e), fg="red")

# Create GUI window
window = tk.Tk()
window.title("CSV Hashing Tool")

# Browse file button and label
browse_button = tk.Button(window, text="Browse", command=browse_files)
browse_button.grid(row=0, column=0, pady=10, padx=10)
path_entry = tk.Entry(window, width=50)
path_entry.grid(row=0, column=1, columnspan=2, pady=10)

# Convert button and status label
convert_button = tk.Button(window, text="Convert", command=convert)
convert_button.grid(row=1, column=0, pady=10, padx=10)
status_label = tk.Label(window, text="")
status_label.grid(row=1, column=1, columnspan=2, pady=10)

# Set window size and prevent resizing
window.geometry("500x150")
window.resizable(False, False)

window.mainloop()
